# Part 2 Instructions
### Narrowing Travel Searches Based on Temperature and Precipitation for Customers

In [38]:
# 1. Dependacies

import numpy as np
import random
import requests
import pandas as pd
from citipy import citipy
import matplotlib.pyplot as plt
import os
import gmaps
from config import g_key
from config import weather_api_key
from datetime import datetime



In [2]:
# Reloading the config since adding new keys

import importlib
import config

importlib.reload(config)

from config import challenge_weather_key

from config import g_key2

In [13]:
# 2. Importing dataframe

city_data_df = pd.read_csv("./challenge_data/WeatherPy_database.csv")

city_data_df.head()

city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp (F)           float64
% Humidity               int64
% Cloudiness             int64
Wind Speed (mph)       float64
Current Description     object
Rain (inches)          float64
Snow (inches)          float64
dtype: object

In [4]:
# Configuring gmaps to use g_key

gmaps.configure(api_key=g_key2)

In [15]:
# 3. Prefernces and Filtering
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_pref = input("Do you want it to be raining? (yes/no) ")
snow_pref = input("Do you want it to be snowing? (yes/no) ")



# Filter dataset to provide a preferred city name according to clients input, including rain and snow


if rain_pref == "no" and snow_pref == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp (F)'] <= max_temp) &
                                      (city_data_df['Max Temp (F)'] >= min_temp) &
                                      (city_data_df['Rain (inches)'] == 0) &
                                      (city_data_df['Snow (inches)'] == 0)]
elif rain_pref == "yes" and snow_pref == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp (F)'] <= max_temp) &
                                      (city_data_df['Max Temp (F)'] >= min_temp) &
                                      (city_data_df['Rain (inches)'] > 0) &
                                      (city_data_df['Snow (inches)'] > 0)]
elif rain_pref == "yes" and snow_pref == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp (F)'] <= max_temp) &
                                      (city_data_df['Max Temp (F)'] >= min_temp) &
                                      (city_data_df['Rain (inches)'] > 0) &
                                      (city_data_df['Snow (inches)'] == 0)]
elif rain_pref == "no" and snow_pref == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp (F)'] <= max_temp) &
                                      (city_data_df['Max Temp (F)'] >= min_temp) &
                                      (city_data_df['Rain (inches)'] == 0) &
                                      (city_data_df['Snow (inches)'] > 0)]
preferred_cities_df

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


,City_ID,City,Country,Lat,Lng,Max Temp (F),% Humidity,% Cloudiness,Wind Speed (mph),Current Description,Rain (inches),Snow (inches)
2,2,Kaka,TM,37.35,59.61,73.18,19,0,6.42,clear sky,0.0,0.0
5,5,Alofi,NU,-19.06,-169.92,75.20,88,89,5.82,overcast clouds,0.0,0.0
8,8,Ormara,PK,25.21,64.64,85.06,82,92,16.06,overcast clouds,0.0,0.0
15,15,Labe,GN,11.32,-12.28,70.00,96,88,4.59,overcast clouds,0.0,0.0
16,16,Kupang,ID,-10.17,123.58,73.83,84,96,8.88,overcast clouds,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
559,559,Motupe,PE,-6.15,-79.71,85.12,40,1,10.27,clear sky,0.0,0.0
560,560,Finschhafen,PG,-6.60,147.87,78.04,78,100,7.63,overcast clouds,0.0,0.0
561,561,Le Pradet,FR,43.11,6.02,77.00,69,0,8.05,clear sky,0.0,0.0
564,564,Porto Santo,PT,33.07,-16.34,71.60,78,75,14.99,broken clouds,0.0,0.0


In [20]:
# 4.  Adding new dataframe to showcase marker layer map for all the nearby hotels in the cities

hotel_df = preferred_cities_df[['City', 'Country','Max Temp (F)','Current Description','Lat','Lng']].copy()

hotel_df.head()

# Entering an empty hotel array

hotel_df["Hotel Name"] = ""

hotel_df

# Finding all the nearby hotels

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key2}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping..zz")



Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz


In [28]:
# Saving file to "data" folder
hotel_df.to_csv("./challenge_data/WeatherPy_vacation.csv")

In [37]:
# Adding the marker label pop-up description

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp (F)} °F</dd>
</dl>
"""


# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]

max_temp = hotel_df["Max Temp (F)"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# heat_layer = gmaps.heatmap_layer(locations,
#                                  weights=max_temp,
#                                  dissipating=False,
#                                  max_intensity=300,
#                                  point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# fig.add_layer(heat_layer)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig



Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Saving the figure
embed_minimal_html('./challenge_image/WeatherPy_vacation_map.html', views=[fig])